In [80]:
%pip install tensorflow==2.5.0 scikit-learn optuna

ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.5.0
Note: you may need to restart the kernel to use updated packages.


In [81]:
import pandas as pd
import numpy as np

In [82]:
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [83]:
import tensorflow as tf

In [84]:
print(tf.__version__)
print(np.__version__)

2.18.0
1.26.4


In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [86]:
def get_mlp_model(input_shape,hidden_layer_one,dropout_one,hidden_layer_two,dropout_two):
    model = Sequential()
    model.add(Dense(hidden_layer_one, activation='elu', input_shape=input_shape))
    if dropout_one !=0:
        model.add(Dropout(dropout_one))
    model.add(Dense(hidden_layer_two, activation='elu'))
    if dropout_two !=0:
        model.add(Dropout(dropout_two))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer="Adam")

    return model

In [87]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    rrse = np.sqrt(np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    mape = np.mean(np.abs((predicted - trues) / trues)) * 100
    r2 = max(0, 1 - np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    # Calculez les autres mesures d'erreur ici
    return rmse, corr, mae, rae, rrse, mape, r2

In [88]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import optuna
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.neural_network import MLPRegressor
from tensorflow.keras.callbacks import ModelCheckpoint

In [89]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

# We start by loading the pre-processed data

In [90]:
order_history_data=pd.read_csv('../data/final_data.csv')
order_history_data.head()

,Inflation,Month,Year,pmi,PartNo,Description,Quantity,Sales Price,Total Sales,Swaraj Engines Stock Price,M&M Stock Price,Escorts Kuboto Stock Price
0,7.59,1,2020,250,0503BA0290N-RK,ALFA HUB,85,2118.60,59367,1125.0,532.9,630.0
1,7.59,1,2020,250,0503BA0290N,Front Wheel Hub,318,3536.35,59367,1125.0,532.9,630.0
2,7.59,1,2020,250,0502BA1050N,Alfa Rear Wheel Hub LH,209,1383.36,59367,1125.0,532.9,630.0
3,7.59,1,2020,250,0502BA1060N,Alfa Rear Wheel Hub RH,119,4849.50,59367,1125.0,532.9,630.0
4,7.59,1,2020,250,0602AAB00260N,Brake Drum ALFA,134,2462.26,59367,1125.0,532.9,630.0


# Deep learning model optimization

In [91]:
X = order_history_data.copy()
y = order_history_data['Quantity']

X_encoded = pd.get_dummies(X, columns=['Description', 'PartNo'])

X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y, test_size=0.3, random_state=27)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=27)

We're going to use mlp. To start with, we'll create an mlp with minimal parameters to get an idea of performance without model optimization. Then we'll optmize our model using Bayesian optimization.

In [92]:

best_mlp = MLPRegressor(hidden_layer_sizes=(50), activation='relu', alpha=0.008724528119026307, learning_rate='constant')

# Train the model on the training data
best_mlp.fit(X_train, y_train)

# Make predictions on the validation set with the trained model
y_val_pred = best_mlp.predict(X_val)

# Evaluate the performance of the model on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = best_mlp.score(X_val, y_val)

# Make predictions on the test set with the trained model
y_test_pred = best_mlp.predict(X_test)

# Evaluate the performance of the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = best_mlp.score(X_test, y_test)

print("Validation Set:")
print("MSE (Mean Squared Error) on the validation set:", mse_val)
print("R-squared (Coefficient of Determination) on the validation set:", r2_val)

print("\nTest Set:")
print("MSE (Mean Squared Error) on the test set:", mse_test)
print("R-squared (Coefficient of Determination) on the test set:", r2_test)



Validation Set:
MSE (Mean Squared Error) on the validation set: 2.659763823422743
R-squared (Coefficient of Determination) on the validation set: 0.9998764321014058

Test Set:
MSE (Mean Squared Error) on the test set: 4.900877106181395
R-squared (Coefficient of Determination) on the test set: 0.9997818790549112


Using Bayesian optimization to find the best hyperparameters

In [93]:
# Objective function for Optuna optimization
def objective(trial):
    # Define the hyperparameters to be optimized
    hidden_layer_one = trial.suggest_categorical('hidden_layer_one', [32])
    hidden_layer_two = trial.suggest_categorical('hidden_layer_two', [64, 128])
    dropout_one = trial.suggest_uniform('dropout_one', 0.28, 0.35)
    dropout_two = trial.suggest_uniform('dropout_two', 0.11, 0.23)
    # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128,256,512])
    epochs = trial.suggest_categorical('epochs', [200])



    model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)


    # Define the ModelCheckpoint callback
    checkpoint_filepath = 'model_checkpoint.h5'
    model_checkpoint = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=0
    )

    # Train the model on the training data with validation data and checkpoint callback
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0
                        , validation_data=(X_val, y_val),
                        callbacks=[model_checkpoint])

    # Load the best weights from the saved checkpoint
    model.load_weights(checkpoint_filepath)

    # Evaluate the performance of the model on the test set
    y_test_pred = model.predict(X_test)

    # Calculate performance metrics
    mse_test = mean_squared_error(y_test, y_test_pred)
    # rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

    return mse_test

# Configure Optuna to use the GPU for exhaustive searches
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=65, n_jobs=1)  # Use n_jobs=1 to avoid parallelism problems on GPU

# Show best hyperparameters found
print("Best hy    perparameters:")
print(study.best_params)
print("Best MSE:", study.best_value)

[I 2025-02-21 19:12:38,298] A new study created in memory with name: no-name-823b9741-0791-4f90-9077-c00afb6211e0


/tmp/ipykernel_3914014/244905401.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0.28, 0.35)
/tmp/ipykernel_3914014/244905401.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0.11, 0.23)
/home/panda/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


[I 2025-02-21 19:12:58,281] Trial 0 finished with value: 1284.0692246578292 and parameters: {'hidden_layer_one': 32, 'hidden_layer_two': 64, 'dropout_one': 0.28871787553030803, 'dropout_two': 0.1288270680298476, 'batch_size': 256, 'epochs': 200}. Best is trial 0 with value: 1284.0692246578292.
/tmp/ipykernel_3914014/244905401.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0.28, 0.35)
/tmp/ipykernel_3914014/244905401.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0.11, 0.23)
/home/panda/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_s

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step


[I 2025-02-21 19:55:17,669] Trial 63 finished with value: 3953.714513702013 and parameters: {'hidden_layer_one': 32, 'hidden_layer_two': 128, 'dropout_one': 0.29119125434694504, 'dropout_two': 0.11233646269636129, 'batch_size': 64, 'epochs': 200}. Best is trial 8 with value: 463.72700309891405.
/tmp/ipykernel_3914014/244905401.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_one = trial.suggest_uniform('dropout_one', 0.28, 0.35)
/tmp/ipykernel_3914014/244905401.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_two = trial.suggest_uniform('dropout_two', 0.11, 0.23)
/home/panda/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_

49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 941us/step


[I 2025-02-21 19:55:56,202] Trial 64 finished with value: 3694.9390018619597 and parameters: {'hidden_layer_one': 32, 'hidden_layer_two': 128, 'dropout_one': 0.29509497996194145, 'dropout_two': 0.1259212040466303, 'batch_size': 64, 'epochs': 200}. Best is trial 8 with value: 463.72700309891405.


Best hy    perparameters:
{'hidden_layer_one': 32, 'hidden_layer_two': 128, 'dropout_one': 0.29778666582173113, 'dropout_two': 0.1156354551254656, 'batch_size': 512, 'epochs': 200}
Best MSE: 463.72700309891405


In [94]:
best_params = study.best_params
best_mse = study.best_value

In [95]:
print(best_params)
print(best_mse)

{'hidden_layer_one': 32, 'hidden_layer_two': 128, 'dropout_one': 0.29778666582173113, 'dropout_two': 0.1156354551254656, 'batch_size': 512, 'epochs': 200}
463.72700309891405


Using Grid Search

In [96]:
# r2=0

# for hidden_layer_one in [25,50,75,100,125,150,175,200,225,350,300,350,400]:
#   for hidden_layer_two in [325,50,75,100,125,150,175,200,225,350,300,350,400]:
#       for dropout_one in [0,0.1,0.2,0.3,0.4,0.5]:
#         for dropout_two in [0,0.1,0.2,0.3,0.4,0.5]:
#           # for batch_size in [32,64,128,256]:


#             model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

#             # Define the ModelCheckpoint callback
#             checkpoint_filepath = 'model_checkpoint.h5'
#             model_checkpoint = ModelCheckpoint(
#                 filepath=checkpoint_filepath,
#                 save_best_only=True,
#                 monitor='val_loss',
#                 mode='min',
#                 verbose=0
#             )

#             # Train the model on the training data with validation data and checkpoint callback
#             history = model.fit(X_train, y_train, epochs=300, batch_size=256,
#                                 verbose=0, validation_data=(X_val, y_val),
#                                 callbacks=[model_checkpoint])

#             # Load the best weights from the saved checkpoint
#             model.load_weights(checkpoint_filepath)

#             # Evaluate the performance of the model on the test set
#             y_test_pred = model.predict(X_test)

#             # Calculate performance metrics
#             mse_test = mean_squared_error(y_test, y_test_pred)
#             rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

#             if r2_2>r2:
#               r2=r2_2
#               print("{},{},{},{}".format(hidden_layer_one,hidden_layer_two,dropout_one,dropout_two) )
#               print("RMSE:", rmse2)
#               print("Corrélation:", corr2)
#               print("MAE:", mae2)
#               print("RAE:", rae2)
#               print("RRSE:", rrse2)
#               print("MAPE:", mape2)
#               print("R2:", r2_2)
#               print("MSE:", mse_test)
#               print("----------------------------------")

Training model with best hyperparamters

In [97]:
# best parameters

hidden_layer_one=best_params['hidden_layer_one']
hidden_layer_two=best_params['hidden_layer_two']
dropout_one=best_params['dropout_one']  
dropout_two=best_params['dropout_two']

epochs=best_params['epochs']
batch_size=best_params['batch_size']


model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

# Define the ModelCheckpoint callback
checkpoint_filepath = '../checkpoints/bayesian_final_model_checkpoint.h5'
model_checkpoint = ModelCheckpoint(
filepath=checkpoint_filepath,
save_best_only=True,
monitor='val_loss',
mode='min',
verbose=0
)

# Train the model on the training data with validation data and checkpoint callback
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
            verbose=2, validation_data=(X_val, y_val),
            callbacks=[model_checkpoint])

# Load the best weights from the saved checkpoint
model.load_weights(checkpoint_filepath)

# Evaluate the performance of the model on the test set
y_test_pred = model.predict(X_test)

# Calculate performance metrics
rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

print("RMSE:", rmse2)
print("Corrélation:", corr2)
print("MAE:", mae2)
print("RAE:", rae2)
print("RRSE:", rrse2)
print("MAPE:", mape2)
print("R2:", r2_2)
print("----------------------------------")

Epoch 1/200


/home/panda/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 - 1s - 44ms/step - loss: 2198241.7500 - val_loss: 144870.4844
Epoch 2/200


15/15 - 0s - 8ms/step - loss: 786002.5000 - val_loss: 29176.4883
Epoch 3/200
15/15 - 0s - 6ms/step - loss: 425309.2188 - val_loss: 29545.9863
Epoch 4/200
15/15 - 0s - 7ms/step - loss: 293231.8125 - val_loss: 52372.9766
Epoch 5/200
15/15 - 0s - 8ms/step - loss: 216753.4688 - val_loss: 42225.5195
Epoch 6/200
15/15 - 0s - 6ms/step - loss: 166892.3125 - val_loss: 50577.1445
Epoch 7/200
15/15 - 0s - 6ms/step - loss: 133287.2500 - val_loss: 45757.3164
Epoch 8/200
15/15 - 0s - 6ms/step - loss: 111813.7266 - val_loss: 40013.4922
Epoch 9/200
15/15 - 0s - 7ms/step - loss: 94618.1328 - val_loss: 38043.9375
Epoch 10/200
15/15 - 0s - 5ms/step - loss: 80453.1797 - val_loss: 32237.2891
Epoch 11/200
15/15 - 0s - 5ms/step - loss: 73440.8438 - val_loss: 32809.6602
Epoch 12/200
15/15 - 0s - 5ms/step - loss: 64202.6719 - val_loss: 30931.7246
Epoch 13/200
15/15 - 0s - 5ms/step - loss: 58596.0508 - val_loss: 29309.6113
Epoch 14/200
15/15 - 0s - 5ms/step - loss: 51936.5664 - val_loss: 29713.7734
Epoch 15/200

15/15 - 0s - 6ms/step - loss: 48849.5859 - val_loss: 28814.6152
Epoch 16/200
15/15 - 0s - 5ms/step - loss: 44448.5352 - val_loss: 29061.6797
Epoch 17/200


...

15/15 - 0s - 7ms/step - loss: 11183.4873 - val_loss: 3238.8447
Epoch 46/200
15/15 - 0s - 5ms/step - loss: 11035.2344 - val_loss: 4358.0308
Epoch 47/200
15/15 - 0s - 6ms/step - loss: 11204.0088 - val_loss: 3716.2957
Epoch 48/200


15/15 - 0s - 6ms/step - loss: 10802.0625 - val_loss: 3157.7649
Epoch 49/200


15/15 - 0s - 7ms/step - loss: 11029.2754 - val_loss: 2632.5562
Epoch 50/200
15/15 - 0s - 6ms/step - loss: 10136.9902 - val_loss: 4209.2598
Epoch 51/200


15/15 - 0s - 6ms/step - loss: 10082.8115 - val_loss: 2480.7517
Epoch 52/200
15/15 - 0s - 5ms/step - loss: 10602.8877 - val_loss: 3252.2336
Epoch 53/200
15/15 - 0s - 6ms/step - loss: 10027.9365 - val_loss: 3746.2122
Epoch 54/200


15/15 - 0s - 6ms/step - loss: 9703.9043 - val_loss: 1513.0594
Epoch 55/200
15/15 - 0s - 6ms/step - loss: 9797.4600 - val_loss: 3144.5081
Epoch 56/200
15/15 - 0s - 5ms/step - loss: 9495.1416 - val_loss: 3610.8792
Epoch 57/200
15/15 - 0s - 5ms/step - loss: 9364.5469 - val_loss: 4718.3013
Epoch 58/200
15/15 - 0s - 7ms/step - loss: 9218.6201 - val_loss: 4635.8940
Epoch 59/200
15/15 - 0s - 5ms/step - loss: 8979.1631 - val_loss: 2465.9739
Epoch 60/200
15/15 - 0s - 5ms/step - loss: 8612.2109 - val_loss: 4186.7417
Epoch 61/200
15/15 - 0s - 5ms/step - loss: 8594.7041 - val_loss: 3218.6191
Epoch 62/200
15/15 - 0s - 5ms/step - loss: 8441.0762 - val_loss: 1883.1222
Epoch 63/200
15/15 - 0s - 5ms/step - loss: 8614.4346 - val_loss: 5531.3984
Epoch 64/200
15/15 - 0s - 5ms/step - loss: 8298.5811 - val_loss: 3840.1711
Epoch 65/200
15/15 - 0s - 5ms/step - loss: 8245.4189 - val_loss: 2060.5432
Epoch 66/200
15/15 - 0s - 5ms/step - loss: 8474.9824 - val_loss: 5096.3730
Epoch 67/200
15/15 - 0s - 8ms/step - l

Save the weights of the best model for later loading

In [98]:
# Save the best model in checkpoint only if it is better
model.save_weights('../checkpoints/bayesian_final.weights.h5')